# 5. 오차역전파

In [5]:
import numpy as np

# 5.4 단순한 계층구현하기
####  계산그래프에서의 곱셈노드('MulLayer'), 덧셈노드 ('AddLayer') 구현하기

In [4]:
# 곱셈 계층

class MulLayer :
    def __init__(self) :
        self.x = None
        self.y = None

    # x와 y를 인수로 받고 두 값을 곱해서 반환.
    def forward(self, x, y) :
        self.x = x
        self.y = y
        out = x * y

        return out
        
    # 상류에서 넘어온 미분(dout)에 순전파때의 값을 서로 바꿔 곱한 후 하류로 흘림.
    def backward(self, dout) :
        dx = dout * self.y    # change x with y
        dy = dout * self.x

        return dx, dy

In [16]:
# 예시) 사과 2개 구입

apple = 100
apple_num = 2
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()


#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print(price)





220.00000000000003


In [17]:
# 역전파
# backward()가 받는 인수는 '순전파의 출력에 대한 미분'

dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print(dapple, dapple_num, dtax)


2.2 110.00000000000001 200


In [33]:
# 덧셈 계층
#backward()에서 상류에서 내려온 미분을 그대로 하류로 흫림.


class AddLayer :
    def __init__(self) :
        pass
        
    def forward(self, x, y) :
        out = x + y
        return out

    def backward(self, dout) :
        dx = dout * 1
        dy = dout * 1 
        return dx, dy





In [34]:
# 예시) 사과 2개와 귤 3개 구입

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1


#계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
mul_fruit_layer = AddLayer()
mul_tax_layer = MulLayer()


#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
fruit_price = mul_fruit_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(fruit_price, tax)

print(price)


715.0000000000001


In [35]:



# 역전파
dprice = 1
dfruit_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = mul_fruit_layer.backward(dfruit_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)



print(dfruit_price, dtax)
print(dapple_price, dorange_price)
print(dapple, dapple_num)
print(dorange, dorange_num)

1.1 650
1.1 1.1
2.2 110.00000000000001
3.3000000000000003 165.0


# 5.5 활성화 함수 계층 구현하기

In [3]:
import numpy as np

In [4]:
class ReLU :
    def __init__(self) :
        self.mask = None

    def forward(self, x) :
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out

    def backward(self, dout) :
        dout[self.mask] = 0
        dx = dout

        return dx        

In [5]:
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [6]:
mask = (x <= 0)
print(mask)

[[False  True]
 [ True False]]


In [7]:
class Sigmoid :
    def __init__(self) :
        self.out = None
        
    def forward(self, x) :
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out

    def backward(self, dout) :
        dx = dout * self.out * (1 - self.out)
        return dx

# 5.6 Affine/Softmax 계층 구현하기

In [8]:
import numpy as np

In [10]:
X = np.random.rand(2)
W = np.random.rand(2,3)
B = np.random.rand(3)

In [12]:
print(X.shape)
print(W.shape)
print(B.shape)


(2,)
(2, 3)
(3,)


In [15]:
Y = np.dot(X,W) + B

In [16]:
X_dot_W = np.array([[0,0,0], [10,10,10]])

In [17]:
B = np.array([1,2,3])

In [18]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [19]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [20]:
dY = np.array([[1,2,3], [4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [22]:
dB = np.sum(dY, axis = 0)
dB

array([5, 7, 9])

In [24]:
# Affine 구현

class Affine :
    def __init__(self, W, b) :
        self.W = W
        self.b = b
        self.x = None
        self.dw = None
        self.db = None
        
    def forward(self, x) :
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, dout) :
        dx = np.dot(dout, self.W.T)
        self.dw = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)

        return dx


In [38]:
# softmax함수 구현

def softmax(a) :
    c = np.max(a)
    exp_a = np.exp(a - c) # 오버플로우 방지
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    
    return y
    
def cross_entropy(y,t) :
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

In [41]:
# softmax_loss 계층을 구현함 
class SoftmaxWithLoss :
    def __init__(self) :
        self.loss = None # 손실
        self.y = None # softmax의 출력
        self.t = None # 정답 레이블(원-핫 벡터)
    
    def forward(self, x, t) :
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy(self.y, self.t)
        return self.loss

    def backward(self, dout = 1) :
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size  # 평균화된 기울기를 구함
        return dx
        


# 5.7 오차역전파 구현

In [1]:
import numpy as np


# 시그모이드 함수구현
def sigmoid(x) :
    return 1 / (1 + np.exp(-x))


# softmax함수 구현
def softmax(a) :
    c = np.max(a)
    exp_a = np.exp(a - c) # 오버플로우 방지
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y


# 크로스 엔트로피 계산
def cross_entropy(y,t) :
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))


# 수치 미분  구현 
def numerical_gradient(f, x):
    h = 1e-7 # 0.0000001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad





In [139]:
# layer들 구현

# Affine 구현
class Affine :
    def __init__(self, W, b) :
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x) :
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, dout) :
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)

        return dx


# ReLU게층 구현함
class ReLU :
    def __init__(self) :
        self.mask = None

    def forward(self, x) :
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        
        return out

    def backward(self, dout) :
        dout[self.mask] = 0
        dx = dout

        return dx 


# softmax_loss 계층을 구현함 
class SoftmaxWithLoss :
    def __init__(self) :
        self.loss = None # 손실
        self.y = None # softmax의 출력
        self.t = None # 정답 레이블(원-핫 벡터)
    
    def forward(self, x, t) :
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy(self.y, self.t)
        return self.loss

    def backward(self, dout = 1) :
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size  # 평균화된 기울기를 구함
        return dx
        

In [140]:
from collections import OrderedDict


# two layer로 구현해보기!
class TwoLayerNet :
    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.1) :
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['ReLU1'] = ReLU()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x) :
        for layer in self.layers.values() :
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t) :
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t) :
        y = self.predict(x) 
        y = np.argmax(y, axis = 1) 
        if t.ndim != 1 : t = np.argmax(t, axis = 1)

        accuracy = np.sum(y == t) / float(x.shape[0])

        return accuracy


    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t) :
        loss_W = lambda W : self.loss(x, t) 

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    

    def gradient(self, x, t) :
        # 순전파 
        self.loss(x, t)

        # 역전파 
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        
        for layer in layers :
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db


        return grads



# 5.7.3 오차역전파법으로 구한 기울기 검증하기 

In [141]:
#입력층은 28*28
#출력층은 10(0~9)
# 첫번째 은닉층은 50개 두번째 은닉층은 100개로 임의로 정함
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# 0으로 채워진 길이 n의 리스트를 반환 
def zeros(n) :
    return [0 for i in range(n)]

# 1차원 넘파이 배열을 2차원 one_hot_enconding으로 반환 
# 주의 근데 여기서만 써야함
def one_hot_encoding_in_one_to_two(y, n) :
    output = []
    for i in y :
        row = zeros(n) 
        row[i] = 1
        output.append(row)
    return np.array(output)

# 주의 4장에서만 사용할 것
def get_data_with_one_hot_encoding() :
    mnist = fetch_openml('mnist_784', version=1, as_frame=False)
    X, y = mnist.data, mnist.target
    y = y.astype(int) 
    X = X.astype(np.float64)  # 나누기를 하기위해 타입변환
    X /= 255.0   #정규화, 0~1값으로 만들기

    y = one_hot_encoding_in_one_to_two(y, 10) # 1차원 넘파이 배열을 2차원 one_hot_enconding으로 반환 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/7, random_state=42)
    
    return (X_train, y_train), (X_test, y_test)



# 5.7.3 오차역전파법으로 구한 기울기 검증하기

In [142]:

# 데이터 준비
(X_train, y_train) , (X_test, y_test) = get_data_with_one_hot_encoding()


In [143]:


network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

X_batch = X_train[100:103]
y_batch = y_train[100:103]
grad_numerical = network.numerical_gradient(X_batch, y_batch)
grad_backprop = network.gradient(X_batch, y_batch)


# 기울기 확인 // 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값의 평균을 낸다.
for key in grad_numerical.keys() :
    diff = (np.average(np.abs(grad_backprop[key] - grad_numerical[key]) ))
    print(key + " : " + str(diff))    



W1 : 0.008178542671676385
b1 : 0.05087470906741442
W2 : 0.1477302128277587
b2 : 0.32414527794242876


# 5.74 오차역전파법을 사용한 학습 구현하기

In [144]:
# 데이터 읽기 
(X_train, y_train) , (X_test, y_test) = get_data_with_one_hot_encoding()

In [145]:

# 모델 데리고 오기
network = TwoLayerNet(input_size = 784, hidden_size = 50, output_size = 10)

iters_num = 100000
train_size = X_train.shape[0]
batch_size = 100 
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)


for i in range(iters_num) :
    batch_mask = np.random.choice(train_size, batch_size)
    X_batch = X_train[batch_mask]
    y_batch = y_train[batch_mask]

    # 오차역전파법으로 기울기를 구한다.
    grad = network.gradient(X_batch, y_batch)

    # 갱신
    for key in ('W1', 'b1', 'W2', 'b2') :
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(X_batch, y_batch)
    train_loss_list.append(loss)


    if i % iter_per_epoch == 0 :
        train_acc = network.accuracy(X_train, y_train)
        test_acc = network.accuracy(X_test, y_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)





0.08076666666666667 0.0827
0.09866666666666667 0.0983
0.10178333333333334 0.1034
0.10038333333333334 0.0967


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14460\2709667214.py:11: RuntimeWarning: invalid value encountered in subtract
  exp_a = np.exp(a - c) # 오버플로우 방지


0.09866666666666667 0.0983
0.09866666666666667 0.0983
0.09866666666666667 0.0983
0.09866666666666667 0.0983
0.09866666666666667 0.0983
0.09866666666666667 0.0983


KeyboardInterrupt: 